# Module Assignment 6
DS 2500 - Data Wrangling  
Professor Marina Kogan  
University of Utah  

Author: Richard Timpson

For this assignment I would like to do a time series analysis of home energy consumption data. This consumption data is a perfect candidate for time series analysis because the data consists of a simple timestamp and associated energy usage value (typically in kWh). My overall goal for this assignment is to get a better idea of what energy home patterns are like and to see how consistent they are across different homes. This analysis will include exploring the different temporal relationships in the usage data (hours vs days vs months) and associated visualizations that show the trends. If possible I would also like to explore the distribution of the data and consider some descriptive statistics. If I have the time to build a predictive model I would like to. 

## Data
I am pulling from a few different data sources that I have access to and that include home usage data at different timestamps for a few different homes. Some of the data I have already gathered from access to a company named SolarEdge and their API. This includes home energy usage for 4 homes. Another data source I will using is homesese.com and can be pulled as a bulk download. I have 2 homes from their data. 

The SolarEdge data comes in 15 minute timestamps whereas the homesense comes in 1 hour. SolarEdge's unit of energy measurement is Wh( watt hour) whereas homesense's unit is kWh(kilowatt hour). homesense needs some initial cleaning while SolarEdge has a lot of missing data. I'll need to resample SolarEdge from 15 minute to 1 hour data, rename columns in both data frames, and convert kWh to Wh so that the dataframes are essentially identical

In [2]:
DATA_DIR = '../../../data/consumption_data'

## Home Sense

In [3]:
import pandas as pd

def read_homesense(site_id):
    dfs = []

    path1 = f'{DATA_DIR}/homesense/{site_id}/jan18-jan19.csv'
    path2 = f'{DATA_DIR}/homesense/{site_id}/jan19-jan20.csv'
    path3 = f'{DATA_DIR}/homesense/{site_id}/jan20-jan21.csv'

    paths = [path1, path2, path3]
    for path in paths:
        # the first row of the csv file is not needed
        df = pd.read_csv(path,skiprows=1)
        dfs.append(df)
    hs_df = pd.concat(dfs)
    return hs_df

In [4]:
hs_dfs = {
    1: read_homesense(1), 
    2: read_homesense(2),
}

This data shows that we have detailed usage information in hourly timestamps for several different devices. There are multiple values for each timestamp, but the one that we are interested in is the Device ID "mains" and name "Total Usage". 

In [5]:
hs_dfs[1]

,DateTime,Device ID,Name,Device Type,Device Make,Device Model,Device Location,Avg Wattage,kWh
0,2018-04-03 19:00:00,mains,Total Usage,NaN,NaN,NaN,NaN,2509.685,2.510
1,2018-04-03 19:00:00,f03206dc,Device 1,Mystery Device,NaN,NaN,NaN,62.409,0.062
2,2018-04-03 20:00:00,mains,Total Usage,NaN,NaN,NaN,NaN,1630.022,1.630
3,2018-04-03 20:00:00,36a3ac21,Microwave,Microwave,NaN,NaN,NaN,14.746,0.015
4,2018-04-03 20:00:00,ac1cccee,Light 1,Light,NaN,NaN,NaN,44.216,0.044
...,...,...,...,...,...,...,...,...,...
21026,2020-04-07 18:00:00,mains,Total Usage,NaN,NaN,NaN,NaN,544.473,0.544
21027,2020-04-07 18:00:00,2593b8f0,Fan,Fan,NaN,NaN,NaN,1.263,0.001
21028,2020-04-07 18:00:00,always_on,Always On,AlwaysOn,NaN,NaN,NaN,268.000,0.268
21029,2020-04-07 18:00:00,d9ca7cf7,Light 4,Light,NaN,NaN,NaN,2.805,0.003


We'll want to filter the data to only have rows with those values. Let's first do a value count to check to make sure that the number of rows with "mains" and "Total Usage" is the same

In [6]:
print('System 1')
print(hs_dfs[1]['Device ID'].value_counts()[:5])

print('System 2')
print(hs_dfs[2]['Device ID'].value_counts()[:5])

System 1
mains        17092
always_on    17051
03eddf75     10730
000f976b     10658
e3eaa9b7      7512
Name: Device ID, dtype: int64
System 2
mains        17608
always_on    17577
6a0788e4     14388
2cd25dbc     12060
fe39b6d0     11649
Name: Device ID, dtype: int64


In [7]:
print('System 1')
print(hs_dfs[1]['Name'].value_counts()[:5])

print('System 2')
print(hs_dfs[2]['Name'].value_counts()[:5])

System 1
Total Usage    17092
Always On      17051
Fridge 2       10730
Light 2        10658
Heat pump       9066
Name: Name, dtype: int64
System 2
Total Usage    17608
Always On      17577
Fridge 4       14388
Fridge 3       12060
Light 2        11649
Name: Name, dtype: int64


Both "mains" and "Total Usage" have the same count, so we can filter rows that have both of these values

In [8]:
for i in [1,2]:
    hs_dfs[i] = hs_dfs[i].loc[(hs_dfs[i]['Device ID'] == "mains") & (hs_dfs[i]['Name'] == "Total Usage")]    

We can see that this data frame has the correct number of rows

In [9]:
for i in [1,2]:
    print(hs_dfs[i].shape)

(17092, 9)
(17608, 9)


We'll now want to clean the data by removing columns that aren't needed, renaming columns, and adjusting the unit to be Wh instead of kWh

In [10]:
def clean_homesense(df):
    df_clean = df.copy()
    df_clean['date'] = pd.to_datetime(df_clean['DateTime'])
    df_clean = df_clean.set_index('date')
    df_clean = df_clean[['Avg Wattage', 'kWh']]
    df_clean['kWh'] = df_clean['kWh'].apply(lambda x: x * 1000)
    df_clean = df_clean.rename(columns={'kWh': 'Wh'})
    return df_clean

In [11]:
hs_dfs_clean = {
    1: clean_homesense(hs_dfs[1]),
    2: clean_homesense(hs_dfs[2])
}

In [12]:
hs_dfs_clean[1]

,Avg Wattage,Wh
date,,
2018-04-03 19:00:00,2509.685,2510.0
2018-04-03 20:00:00,1630.022,1630.0
2018-04-03 21:00:00,1104.002,1104.0
2018-04-03 22:00:00,838.236,838.0
2018-04-03 23:00:00,920.732,921.0
...,...,...
2020-04-07 14:00:00,469.358,469.0
2020-04-07 15:00:00,443.339,443.0
2020-04-07 16:00:00,499.031,499.0


In [13]:
hs_dfs_clean[2]

,Avg Wattage,Wh
date,,
2018-04-03 19:00:00,1149.585,1150.0
2018-04-03 20:00:00,659.313,659.0
2018-04-03 21:00:00,821.521,822.0
2018-04-03 22:00:00,3266.778,3267.0
2018-04-03 23:00:00,1984.160,1984.0
...,...,...
2020-04-08 16:00:00,1246.423,1246.0
2020-04-08 17:00:00,2000.381,2000.0
2020-04-08 18:00:00,2979.244,2979.0


In [14]:
print(pd.isnull(hs_dfs_clean[1]['Wh']).values.any())
print(pd.isnull(hs_dfs_clean[2]['Wh']).values.any())

False
False


We can see that there are no zero rows for both data sets, which is promising

## Solar Edge
There are 4 different datasets that come from SolarEdge's api. They consist purely of 15 minute timestamps and associated Wh(watt hour) energy readings. I'll keep all of the SolarEdge data frames in a dictionary. The id's associated with the systems are 466851, 896164, 1520756, 1520780

In [15]:
ids = [466851, 896164, 1520756, 1520780]
se_data = {}
for site_id in ids:
    df = pd.read_csv(f'{DATA_DIR}/SolarEdge/{site_id}/consumption_data.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')

    # this is key as we are resampling the data to 1 hour timestamps instead of 15 minute. Should 
    # make for a more consistent analysis across data sets
    df = df.resample('1H').sum()
    df = df.rename(columns={'value': 'Wh'})
    se_data[site_id] = df


One system has data for 3 years, one roughly 2, and the other only have 2 months worth of data. Some of the systems have a significant amount of null values in the data. I'm not sure how this will affect the analysis but it is important to track this. The best data from the SolarEdge systems is id 896164

In [16]:
def print_site_info(site_id):
    print(f'Site: {site_id}')
    print(f'DF shape: {se_data[site_id].shape}')
    print(f'Number of 0 values: {(se_data[site_id]["Wh"] == 0).sum()}')
    print(f'Number of null values: {pd.isnull(se_data[site_id]["Wh"]).sum()}')

In [17]:
se_data[466851]

,Wh
date,
2017-02-24 00:00:00,0.0
2017-02-24 01:00:00,0.0
2017-02-24 02:00:00,0.0
2017-02-24 03:00:00,0.0
2017-02-24 04:00:00,0.0
...,...
2020-03-23 19:00:00,731.0
2020-03-23 20:00:00,812.0
2020-03-23 21:00:00,748.0


In [18]:
for site_id in ids:
    print_site_info(site_id)

Site: 466851
DF shape: (26976, 1)
Number of 0 values: 10593
Number of null values: 0
Site: 896164
DF shape: (12408, 1)
Number of 0 values: 9
Number of null values: 0
Site: 1520756
DF shape: (696, 1)
Number of 0 values: 522
Number of null values: 0
Site: 1520780
DF shape: (696, 1)
Number of 0 values: 523
Number of null values: 0


# Visualizations
These visualizations are going to be tricky because some homes have a vastly larger number of data points than others. My guess is that the home usage data should be fairly consistent across days with patterns of usage at night and patterns of usage during the day. I would also guess that there are going to be outliers to these patterns that are essentially impossible to predict. Because my overall goal is going to be in building a predictive model I think it is first going to be useful to gain an idea of what the patterns of usage look like at different granularities. I'll list them out.  

1. Seasonally: This might include the average monthly usage across the different years. The hypothesis might be that the times of year that are the hottest and coldest will require the most energy because of central heating and cooling. However, some homes use gas rather than electricity for some appliances so it may not stay the same across all of the homes. It would be useful to compare years to each other to see how much it changes per home, if it changes at all 
2. Daily: What does the home usage look like on a daily basis? Does the usage per day stay consistent or have large variation? Obviously this is going to be affected by the season of the year so there should be a trend that the daily pattern follows (slowly increasing or slowly decreasing). 
3. Hourly: Is more energy used at night than at day or vise versa? How much variation is there in usage by day? This will obviously be affected by the season of the year as the days get longer/shorter so it may be hard to tell from just this data. If I have time I would like to find an api that will give me the sunrise and sunset times of each day (or find a way to calculate it myself) to see what this trend looks like. 

The first thing I would like to do is to give a crude visualization of each system so that we can get an idea of what the data looks like. One of the homes had a significant amount of 0 values so it may not even worth be considering in the analysis. The visualization should give us an idea of how to handle that. 

In [32]:
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [ ]:
def plot_crude(df, site_id):
    df = df.copy()
    f,ax = plt.subplots(1,1,figsize=(12,4))
    plt.ylabel('Wh')
    plt.title(f'System: {site_id}')
    df['Wh'].plot(lw=3,ax=ax)

In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    plot_crude(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    plot_crude(value, key)

The above plots are not that representative of the global pattern in the datasets. I'm going to plot the daily average consumption on top of the data which will give a better understanding of the trend. 

In [ ]:
def plot_rolling(df, site_id):
    df = df.copy()
    f,ax = plt.subplots(1,1,figsize=(12,4))
    df['rolling_mean_wh'] = df['Wh'].rolling(24).mean()
    plt.ylabel('Wh')
    plt.title(f'System: {site_id}')
    df['rolling_mean_wh'].plot(ax=ax, lw=3, color='k')
    df['Wh'].plot(lw=3,ax=ax, color='k', alpha=.5)
    # ax.set_ylim((2e6,6e6))



In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    plot_rolling(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    plot_rolling(value, key)

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

## Seasonal
Here I would like to view the monthly consumption of each home. I'll do two graphs for this. One will be the monthly consumption for the entire length of the system. The other will be monthly consumption with a comparison by year. 

In [ ]:
def plot_monthly_total(df, site_id):
    df = df.copy()
    df = df.resample('1M').sum()
    f,ax = plt.subplots(1,1,figsize=(12,4))
    plt.ylabel('Wh')
    plt.title(f'System: {site_id}')
    df['Wh'].plot(lw=3,ax=ax)


In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    plot_monthly_total(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    plot_monthly_total(value, key)

In [ ]:
import seaborn as sb

def plot_monthly_comparison(df, site_id):
    df = df.copy()
    df = df.resample('1M').sum()
    df['month'] = df.index.month
    df['year'] = df.index.year
    sb.catplot(x='month',y='Wh',hue='year',data=df,kind='point',aspect=2)
    plt.title(f'System: {site_id}')

In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    plot_monthly_comparison(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    plot_monthly_comparison(value, key)

These results are interesting because they show that the usage is not consistent across years. 

## Daily
I'm going to produce the same graphs as above but on a daily sum instead of monthly sum

In [ ]:
def plot_daily_comparison(df, site_id):
    df = df.copy()
    # f,ax = plt.subplots(1,1,figsize=(16,4))
    df = df.resample('1D').sum()
    df['day'] = df.index.dayofyear
    df['year'] = df.index.year
    sb.catplot(x='day',y='Wh',hue='year',data=df,kind='point',height=4, aspect=5)
    plt.title(f'System: {site_id}')

In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    plot_daily_comparison(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    plot_daily_comparison(value, key)

This shows the same results from the monthly graph but on a more granular level. 

## Hourly
Here I would like to see the trends for home usage at different hours of the day. One graph wil just be the average production per hour. 

In [ ]:
def plot_hourly_avg(df, site_id):
    # f,ax = plt.subplots(1,1,figsize=(16,4))
    df = df.copy()
    df['hour'] = df.index.hour
    groups = df.groupby('hour').mean()
    f,ax = plt.subplots(1,1,figsize=(12,4))
    plt.ylabel('Wh')
    plt.title(f'System: {site_id}')
    groups['Wh'].plot(lw=3,ax=ax)

In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    plot_hourly_avg(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    plot_hourly_avg(value, key)

Here I am plotting the average hourly production and comparing across months. Different times of the year should have an affect on the hourly production both because of changes in temperature but also somewhat because of the change in length of days. 

In [ ]:
def plot_hourly_comparison(df, site_id):
    # f,ax = plt.subplots(1,1,figsize=(16,4))
    df = df.copy()
    df['hour'] = df.index.hour
    df['month'] = df.index.month
    sb.catplot(x='hour',y='Wh',hue='month',data=df,kind='point',height=8, aspect=3, ci=None)
    plt.title(f'System: {site_id}')

In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    plot_hourly_comparison(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    plot_hourly_comparison(value, key)

The above graphs are a little bit dense and difficult to interpret (although they are still useful). A related plot I would like to show is the average hourly production and a comparison across seasons of the year. Winter is season 1, Spring season 2, and so forth

In [ ]:
def plot_hourly_comparison_season(df, site_id):
    df = df.copy()
    # f,ax = plt.subplots(1,1,figsize=(16,4))
    df['hour'] = df.index.hour
    # (temp2.dt.month%12 + 3)//3
    df['season'] = (df.index.month%12 + 3) // 3
    sb.catplot(x='hour',y='Wh',hue='season',data=df,kind='point',height=4, aspect=5, ci=None)
    plt.title(f'System: {site_id}')

In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    plot_hourly_comparison_season(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    plot_hourly_comparison_season(value, key)

All of these results show me that the variation in usage across the different homes is large. I was interested to see if there would emerge any global pattern among the usage data but so far that has not been the case. 

# Modelling the Data 
After a visual look at the data I am understanding that usage data is not consistent across homes (at least given the data we have here), but that each home has it's own pattern. I would like to build a predictive model of each home to see how well usage can be predicted just given the temporal pattern.  

## Simple Linear Regression
To start I would like to perform a simple linear regression model using the different temporal. After viewing the above plots, my guess is that just using the hour of the day and the month of the year should produce a somewhat accurate model. 

In [ ]:
def simple_lin_regression(df, site_id):
    print(f"Regression for site: {site_id}")
    df = df.copy()
    df['month'] = df.index.month 
    df['hour'] = df.index.hour
    m2 = smf.ols('Wh ~ C(hour) + C(month)',data=df).fit()
    print(m2.summary())

In [ ]:
print("Plotting SolarEdge data")
for key,value in se_data.items(): 
    simple_lin_regression(value, key)

print("Plotting homesense data")
for key,value in hs_dfs_clean.items(): 
    simple_lin_regression(value, key)

Most of the R-squared values are substantially low so this tells me that these variables do not in fact do a good job of explaining the data. Some of the p-values for the t stastitic in the coeeficients are too high to be statistically significant so that is also to be aware of. 

It's also important to keep in mind that the data for 3 of the systems is extremely flawed. One of the systems has missing data for a year, while 2 of them only have data for a few months. I'm ignoring this here but will take it into account when I build a more sophisticated model. 

## Autocorrelation and Partial Autocorrelation
Given that the linear regression model with the temporal timescales did not do a good job of explaining the data I'll move to dutocorrelation. The first thing that I would like to do is produce some graphs that show the autocorrelation and the incorporate that into a predictive model

In [ ]:
def auto_correlation(df, site_id):
    df = df.copy()
    f,axs = plt.subplots(2,1,figsize=(8,8))
    fig1 = sm.graphics.tsa.plot_acf(df['Wh'],zero=False,lags=30,ax=axs[0],alpha=.05)
    axs[0].set_title(f'Autocorrelation: System {site_id}')
    fig2 = sm.graphics.tsa.plot_pacf(df['Wh'],zero=False,lags=30,ax=axs[1],alpha=.05)
    axs[1].set_title(f'Partial Autocorrelation: System {site_id}')

    # for ax in axs:
    #     ax.axvline(12,c='r',ls='--',lw=1
    #     ax.axvline(24,c='r',ls='--',lw=1)
    #     ax.axvline(36,c='r',ls='--',lw=1)
    #     ax.axvline(48,c='r',ls='--',lw=1)

In [ ]:
for key,value in se_data.items(): 
    auto_correlation(value, key)

for key,value in hs_dfs_clean.items(): 
    auto_correlation(value, key)

These graphs show that the first few lags are heavily correlated with each other and at a high level of stastical significance. This is a good sign for a predictive model. 

In [ ]:
def pd_auto_correlation(df, site_id):
    df = df.copy()
    f,ax = plt.subplots(1,1,figsize=(8,4))

    ax = pd.plotting.autocorrelation_plot(df['Wh'],ax=ax)
    plt.title(f'Auto correlation: System {site_id}')
    # ax.set_xlim((1,50))

In [ ]:
for key,value in se_data.items(): 
    pd_auto_correlation(value, key)

for key,value in hs_dfs_clean.items(): 
    pd_auto_correlation(value, key)

These graphs are interesting because they are showing a cycle in the correlation the greater the lag. This indicates that the hour of the day plays a role in the correlation. 

# Autoregressive Model
As was shown in class, we are going to be using the prophet library to build an auto regressive model for the prediction. We'll have to do some column renaming of our data frames to be compatable with the library. 

In [26]:
pd.plotting.register_matplotlib_converters()
from fbprophet import Prophet

In [27]:
def rename_df_columns(df, site_id):
    df = df.copy()
    df = df.reset_index()
    df = df.rename(columns={"date": "ds", "Wh":"y" })
    df = df[['ds', 'y']]
    return df

In [28]:
# creating one dictionary to hold all data. Don't know why I didn't do this earlier
p_data = {}

for key,value in se_data.items(): 
    df = rename_df_columns(value, key)
    p_data[key] = df

for key,value in hs_dfs_clean.items(): 
    df = rename_df_columns(value, key)
    p_data[key] = df

## More Plots
Prophet makes plotting the data to understand the autoregressive models extremely simple. I'm going to build a forecast model for 30 days and use their plotting functions to show what the trends look like for each system

In [29]:
def run_prophet(df):
    m = Prophet().fit(df)
    m_future = m.make_future_dataframe(periods=30, freq='D')
    m_forecast = m.predict(m_future)
    return m, m_forecast 

def plot_prophet(m, m_forecast, site_id):
    # plot
    f,ax = plt.subplots(1,1,figsize=(10,5))
    _ = m.plot(m_forecast,ax=ax)

    # Always label your axes
    ax.set_xlabel('Time')
    ax.set_ylabel('Wh')
    ax.set_title(f'Forecasted consumption for system {key}')

    m.plot_components(m_forecast)

In [30]:
# df = p_data[1]
# m, m_forecast = run_prophet(df)
# plot_prophet(m, m_forecast, 1)

for key, value in p_data.items():
    m, m_forecast = run_prophet(value)
    plot_prophet(m, m_forecast, site_id)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


NameError: name 'plt' is not defined

## Statistical Results - Cross Validation
All of these plots show that the overall trend of the predictions follows the observations, but there are still large variations between the prediction and the result. The plots are great, but I would like to see a statistical analysis of the error. Prophet has a mechanism for doing that using cross validation. 

In [ ]:
from fbprophet.diagnostics import cross_validation

def fbp_cross_validation(m, site_id):
    df_cv = cross_validation(m, horizon='1 days', initial='35 days', period='1 days')
    print(df_cv.head())

df = p_data[466851]
m, m_forecast = run_prophet(df)
fbp_cross_validation(m, 466851)

In [23]:
import datetime 

d1 = datetime.datetime(day=1, month=5, year = 2018)
d2 = datetime.datetime(day=1, month=1, year=2020)

print(d2 - d1)

610 days, 0:00:00
